## A GAN (Variational AutoEncoders VAE) based Recommendation system

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [ ]:
HOME_DIR = Path("drive/MyDrive/Recys-Project/New-data/combined_files.parquet")

In [ ]:
def get_dataset(location):
  """
  Read the dataset and return just the vehicles with about 20 reviews
  (Majorly to save memory instead of reading in all the rows)
  """
  df = pd.read_parquet(HOME_DIR)
  N_REVIEW = 20
  ids = df.vehicle.value_counts()
  vehicles_N_plus = ids[ids > N_REVIEW].index
  df = df[df.vehicle.isin(vehicles_N_plus)]
  df["vehicle_id"] = pd.factorize(df["vehicle"])[0]
  df = df[["user_id", "vehicle_id", "rating"]]
  return df.reset_index(drop=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = get_dataset(HOME_DIR)

In [ ]:
data.head()

user_id  vehicle_id  rating
0     6467           0     1.0
1     9961           1     5.0
2     2037           1     1.0
3    12983           0     5.0
4      740           0     4.0

In [ ]:
data.shape

(5417, 3)

In [ ]:
num_items = data['vehicle_id'].nunique()
num_users = data['user_id'].nunique()


In [ ]:
num_items, num_users

(192, 4861)

In [ ]:
item_dict = {}
count = 0
for item_id in data['vehicle_id'].unique():
    item_dict[item_id] = count
    count += 1


In [ ]:
user_dict = {}
count = 0
for user_id in data['user_id'].unique():
    user_dict[user_id] = count
    count += 1


In [ ]:
data

user_id  vehicle_id  rating
0        6467           0     1.0
1        9961           1     5.0
2        2037           1     1.0
3       12983           0     5.0
4         740           0     4.0
...       ...         ...     ...
5412     6280         191     5.0
5413      787         191     2.0
5414    18432         191     5.0
5415    17895         191     5.0
5416     3606         191     5.0

[5417 rows x 3 columns]

In [ ]:
# Creating a user-item iteraction matrix with dimension m, n
# rows = users and columns = items (vehicles)
R = np.zeros((num_users, num_items))
for row in data.itertuples():
    R[user_dict[row[1]], item_dict[row[2]]] = row[3]

In [ ]:
R.shape

(4861, 192)

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_size, latent_size):
        super(VAE, self).__init__()
        self.input_size = input_size
        # Encoder
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3_mu = nn.Linear(256, latent_size)
        self.fc3_logvar = nn.Linear(256, latent_size)
        
        # Decoder
        self.fc4 = nn.Linear(latent_size, 256)
        self.fc5 = nn.Linear(256, 512)
        self.fc6 = nn.Linear(512, input_size)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = self.fc3_mu(x)
        logvar = self.fc3_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        z = F.relu(self.fc4(z))
        z = F.relu(self.fc5(z))
        x_hat = torch.sigmoid(self.fc6(z))
        return x_hat

    def forward(self, x):
        x = x.float()
        mu, logvar = self.encode(x) 
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar


In [ ]:
vae = VAE(num_items, latent_size=112)

In [ ]:
vae

VAE(
  (fc1): Linear(in_features=192, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3_mu): Linear(in_features=256, out_features=112, bias=True)
  (fc3_logvar): Linear(in_features=256, out_features=112, bias=True)
  (fc4): Linear(in_features=112, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=192, bias=True)
)

In [ ]:
# Load the data
class CFData(Dataset):
    def __init__(self, data, mask):
        self.data = data
        self.mask = mask

    def __getitem__(self, index):
        x = self.data[index]
        mask = self.mask[index]
        return x, mask
        #return torch.tensor(x, dtype=torch.double), torch.tensor(mask, dtype=torch.double)

    def __len__(self):
        return self.data.shape[0]

In [ ]:
def generate_mask(R, p):
    """
    A mask for the inputs with R as user-item interaction matrix and p is percentage
    of R to mask
    """
    mask = np.random.choice([0, 1], size=R.shape, p=[p, 1-p])
    return R * mask

mask = generate_mask(R, 0.25)

In [ ]:
mask.shape

(4861, 192)

In [ ]:
import torch.nn as nn

class CollaborativeFilteringLoss(nn.Module):
    def __init__(self):
        super(CollaborativeFilteringLoss, self).__init__()
        self.loss_fn = nn.MSELoss()

    def forward(self, predicted_ratings, true_ratings, mask):
        # Flatten the predictions, true ratings and mask tensors
        predicted_ratings = predicted_ratings.view(-1)
        true_ratings = true_ratings.view(-1)
        mask = mask.view(-1)

        # Apply the mask to remove entries that have no ratings
        predicted_ratings = predicted_ratings * mask
        true_ratings = true_ratings * mask

        # Compute the MSE loss between the predicted and true ratings
        loss = self.loss_fn(predicted_ratings, true_ratings)

        return loss


In [ ]:
HOME_DIR.parent

PosixPath('drive/MyDrive/Recys-Project/New-data')

In [ ]:
import os

In [ ]:
class Earlystopping:
  """ stops the model ones it isn't getting any better"""
  def __init__(self, patience = 5, verbose = False, delta = 0):

    """ patience: how long the model should wait to see if the model is improving
        verbose: boolean to print message whenever validatio loss reduce
        delta: minimum change to occur in quantified quantity to know change has occurred"""

    self.patience = patience
    self.verbose = verbose
    self.delta = delta
    self.counter = 0
    self.best_score = None
    self.early_stop = False
    self.val_loss_min = np.Inf

  def __call__(self, val_loss, model):
    score = -val_loss

    if self.best_score is None:
      self.best_score = score 
      self.save_checkpoint(val_loss, model)
    elif score < self.best_score + self.delta:
      self.counter += 1
      print(f"Early stopping counter: {self.counter} out of {self.patience}")
      if self.counter >=self.patience:
        self.early_stop = True

    else:
      self.best_score = score 
      self.save_checkpoint(val_loss, model)
      self.counter = 0

  def save_checkpoint(self, val_loss, model):
    """"saves model when the validation loss decreases"""
    if self.verbose:
      print(f"validation loss decreased from ({self.val_loss_min:.6f} -->  {val_loss:.6f}.   saving model......")
    if os.path.exists(HOME_DIR.parent/"models") == False:
      os.mkdir(HOME_DIR.parent/"models")
    torch.save(model.state_dict(), HOME_DIR.parent/ "models/best_model.pth")
    self.val_loss_min = val_loss

In [ ]:
optimizer = optim.Adam(vae.parameters(), lr=0.001)
early_stopping = Earlystopping(patience= 20, verbose = True)

# Define the training loop
def train_vae(train_loader, test_loader, model, optimizer, criterion, epochs):
    train_loss_list, val_loss, val_accuracy = [],[],[]
    for epoch in range(epochs):
        train_loss = 0
        for i, data in enumerate(train_loader, 0):
            inputs, mask = data
            optimizer.zero_grad()
            outputs, mu, logvar = model(inputs)
            loss = criterion(outputs, inputs, mask)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_loss_list.append(loss)

        valid_accuracy = 0
        test_loss = 0
        with torch.no_grad():
            for i, data in enumerate(test_loader, 0):
                inputs, mask = data
                outputs, mu, logvar = model(inputs)
                loss = criterion(outputs, inputs, mask)
                test_loss += loss.item()
            test_loss /= len(test_loader)
            val_loss.append(test_loss)
            val_accuracy.append(valid_accuracy) 
            early_stopping(test_loss, model)
            print("_"*100)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch}.............")
                break

        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch+1, epochs, train_loss, test_loss))



In [ ]:
# dataset = InteractionDataset(R)
dataset = CFData(R, mask)


# split the dataset into train and test sets
train_size = int(0.8 * len(dataset)) # use 80% of the data for training
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

# create data loaders for training and testing
batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
loss_function = CollaborativeFilteringLoss()


In [ ]:
train_vae(train_loader, test_loader, vae, optimizer, loss_function, epochs=1000)

validation loss decreased from (inf -->  0.967824.   saving model......
____________________________________________________________________________________________________
Epoch [1/1000], Train Loss: 0.9913, Test Loss: 0.9678
validation loss decreased from (0.967824 -->  0.952916.   saving model......
____________________________________________________________________________________________________
Epoch [2/1000], Train Loss: 0.9922, Test Loss: 0.9529
Early stopping counter: 1 out of 20
____________________________________________________________________________________________________
Epoch [3/1000], Train Loss: 0.9920, Test Loss: 0.9554
Early stopping counter: 2 out of 20
____________________________________________________________________________________________________
Epoch [4/1000], Train Loss: 0.9913, Test Loss: 0.9610
Early stopping counter: 3 out of 20
____________________________________________________________________________________________________
Epoch [5/1000], Train 